In [13]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import os
import re
import ast

In [14]:
def fields_for_id_x(field_id):
    field_id = str(field_id)
    field_items = re.split(r'[,\s_]+', field_id)
    if len(field_items) == 1:
        fields = 'p{}'.format(field_items[0])
    elif len(field_items) == 2:
        fields = 'p{}_i{}'.format(field_items[0], field_items[1])
    return fields

import pandas as pd
from ast import literal_eval as safe_literal_eval
import numpy as np

def extract_strings(target_prefixes, icd_codes):
    """
    Extract ICD-10 codes that start with any of the target prefixes.
    
    Parameters:
    - target_prefixes (list): List of prefixes to match.
    - icd_codes (list): List of ICD-10 codes.

    Returns:
    - list: List of matching ICD-10 codes.
    """
    return [code for code in icd_codes if any(code.startswith(prefix) for prefix in target_prefixes)]


In [15]:
def kdm_calc(data, biomarkers, fit=None):
    """
    Calculate Klemera-Doubal Method Biological Age (KDM-BA).

    Parameters:
    - data: pandas DataFrame with biomarker columns and 'age' column
    - biomarkers: list of biomarker column names
    - fit: dict with 'slope' and 'intercept' for each biomarker (optional)

    Returns:
    - DataFrame with KDM-BA and advancement (KDM-BA - chronological age)
    """
    if fit is None:
        # Default fit parameters (example values, should be trained on NHANES III)
        fit = {
            bm: {'slope': 0.1, 'intercept': np.mean(data[bm]), 'sd': np.std(data[bm])}
            for bm in biomarkers
        }

    # Calculate KDM-BA
    kdm_ba = []
    for _, row in data.iterrows():
        age = row['age']
        num = 0
        denom = 0
        for bm in biomarkers:
            x = row[bm]
            k = fit[bm]['slope']
            q = fit[bm]['intercept']
            s = fit[bm]['sd']
            # KDM formula: (biomarker - intercept) / slope contributes to age
            num += ((x - q) / s) * (k / s)
            denom += (k / s) ** 2
        # Final KDM-BA: weighted average of biomarker contributions
        kdm = age + (num / denom) if denom != 0 else age
        kdm_ba.append(kdm)

    result = data.copy()
    result['kdm_ba'] = kdm_ba
    result['kdm_acceleration'] = result['kdm_ba'] - result['age']
    return {'data': result, 'fit': fit}


import pandas as pd
import numpy as np
from lifelines import CoxPHFitter
from scipy.stats import norm

def phenoage_calc(data, biomarkers, fit=None, orig=True):
    """
    Project Phenotypic Age algorithm onto new data or train a new model.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        A dataset containing age, survival time, status, and biomarker columns.
    biomarkers : list
        List of column names for biomarkers used in Phenotypic Age calculation.
        Expected: ['Albumin', 'Alkaline_phosphatase', 'Creatinine', 'C-reactive_protein', 
                   'Glucose', 'White_blood_cell_count', 'Lymphocyte', 'Mean_cell_volume', 
                   'Red_cell_distribution_width']
    fit : lifelines.CoxPHFitter or None
        Pre-trained Cox model fit. If None, a new model is trained.
    orig : bool
        If True, compute the original Phenotypic Age as per Levine 2018.
    
    Returns:
    --------
    dict
        A dictionary with two keys:
        - 'data': DataFrame with added 'phenoage' and 'phenoage_advance' columns.
        - 'fit': Trained CoxPHFitter model (if fit was None) or the provided fit.
    """
    # Copy input data to avoid modifying the original
    data = data.copy()
    
    # Define mapping from provided biomarker names to internal names
    biomarker_mapping = {
        'Albumin': 'albumin',
        'Alkaline_phosphatase': 'alp',
        'Creatinine': 'creat',
        'C-reactive_protein': 'lncrp',  # Will apply log transformation
        'Glucose': 'glucose',
        'White_blood_cell_count': 'wbc',
        'Lymphocyte': 'lymph',
        'Mean_cell_volume': 'mcv',
        'Red_cell_distribution_width': 'rdw'
    }
    
    # Verify all provided biomarkers are valid
    if not all(bm in biomarker_mapping for bm in biomarkers):
        raise ValueError("Invalid biomarker names provided. Expected: " + str(list(biomarker_mapping.keys())))
    
    # Ensure required columns exist
    required_cols = ['age', 'time', 'status'] + biomarkers
    if not all(col in data.columns for col in required_cols):
        raise ValueError("Data must contain 'age', 'time', 'status', and all specified biomarkers")
    
    # Map biomarker names and apply log transformation for C-reactive protein
    internal_biomarkers = [biomarker_mapping[bm] for bm in biomarkers]
    data_mapped = data.copy()
    if 'C-reactive_protein' in biomarkers:
        data_mapped['lncrp'] = np.log(data['C-reactive_protein'] + 1)  # Log transform with offset to handle zeros
    for bm, internal_bm in biomarker_mapping.items():
        if bm in biomarkers and internal_bm != 'lncrp':
            data_mapped[internal_bm] = data[bm]
    
    # If no fit is provided, train a Cox model
    if fit is None:
        # Prepare data for Cox model
        model_data = data_mapped[internal_biomarkers + ['time', 'status']].dropna()
        
        # Fit Cox proportional hazards model
        cph = CoxPHFitter()
        cph.fit(model_data, duration_col='time', event_observed_col='status')
        fit = cph
    else:
        # Use provided fit
        cph = fit
    
    # Compute mortality score (linear predictor from Cox model)
    mortality_score = cph.predict_partial_hazard(data_mapped[internal_biomarkers]).values
    
    # Convert mortality score to Phenotypic Age
    if orig:
        # Parameters from Levine 2018 (supplementary methods)
        gamma = 0.0072973525664  # Gompertz shape parameter
        lambda_ = 0.0000201116   # Gompertz scale parameter
        
        # Compute CDF(120, x) for mortality score
        cdf_120 = 1 - np.exp(-lambda_ * np.exp(mortality_score / gamma))
        
        # Solve for age where CDF_univariate(120, age) = CDF(120, x)
        phenoage = -np.log(1 - cdf_120) / lambda_
    else:
        # For modified Phenotypic Age, use custom parameters
        phenoage = mortality_score  # Simplified; adjust based on specific needs
    
    # Add Phenotypic Age to the dataset
    data['phenoage'] = phenoage
    data['phenoage_advance'] = data['phenoage'] - data['age']
    
    return {'data': data, 'fit': fit}

In [16]:
data_dir = '/Users/xiaoqianxiao/UKB/data'
participantsInfo_file = 'participants.csv'
participantsInfo_file_path = os.path.join(data_dir,participantsInfo_file)
data_df = pd.read_csv(participantsInfo_file_path)

In [17]:
# Biological age factors:
biologicalAge_fields = ['3063_i0', '4080_i0', '30690_i0', '30750_i0', 
                        '30670_i0', '30180_i0', '30270_i0', '30740_i0', 
                        '30070_i0', '30000_i0', '30600_i0', '30700_i0', '30710_i0', '30610_i0']
#demographic factors' field ids, including: eid, sex, age at the first scan, IQ and EA (5)
demographic_fields = ['31','21003','20016','6138']
# Neuroticism fields
neuroticism_fields = ['1920', '1930', '1940', '1950', '1960', '1970', '1980', '1990', 
                      '2000', '2010', '2020', '2030']
# anxiety status files:
anxiety_status_fields = ['1970','1980','1990','2070']
# current depression or anxiety status while scanning (3)
current_status_fields = ['2050','2060','2070']
# Self_Reported_Mental_Health (1)
Self_Reported_Mental_Health_fields = ['29000','20002','21062']
# Ever_Diagnosed_Mental_Health_Problem (1)
Ever_Diagnosed_Mental_Health_Problem_fields = ['20544']
# Self reported history depression: CIDI (13)
history_depression_fields = ['20436','20439','20440','20446','20441','20449','20536','20532','20435','20450','20437']
# Self reported history anxiety: CIDI (18)
history_anxiety_fields = ['20421','20420','20538','20425','20542','20543','20540','20541','20539','20537','20418','20426','20423','20429','20419','20422','20417','20427']
# PHQ (9)
PHQ_fields = ['20514','20510','20517','20519','20511','20507','20508','20518','20513']
# GAD7 (7)
GAD7_fields = ['20506','20509','20520','20515','20516','20505','20512']
GAD7_followup_fields = ['28735','29059','29060','29061','29062','29063','29064']
# hospital data: ICD10 and ICD9 (2)
hospital_data_fields = ['41270','41271']
# fMRI data
fMRI_fields = ['31016','31018','31019','31015','31014']
control_fields = ['20544','20002','20514','20510','20517','20519','20511','20507','20508','20518','20513', '20506','20509','20520','20515','20516','20505','20512']
all_fields_ids = biologicalAge_fields + demographic_fields + current_status_fields + Ever_Diagnosed_Mental_Health_Problem_fields + history_depression_fields + history_anxiety_fields + PHQ_fields + GAD7_fields + GAD7_followup_fields + hospital_data_fields + control_fields + fMRI_fields + Self_Reported_Mental_Health_fields + anxiety_status_fields + neuroticism_fields
print("Number of  all fields needed: ", len(all_fields_ids))

Number of  all fields needed:  118


In [18]:
biologicalAge_fields = ['3063', '4080', '30690', 
                        '30750', '30670', '30180', 
                        '30270', '30740', '30070', 
                        '30000', '30600', '30700', 
                        '30710', '30610']
biologicalAge_names = ['FEV1', 'SBP', 'Cholesterol', 
              'Glycated_hemoglobin', 'Blood_urea_nitrogen', 'Lymphocyte', 
              'Mean_cell_volume', 'Glucose', 'Red_cell_distribution_width',
              'White_blood_cell_count', 'Albumin', 'Creatinine', 
              'C-reactive_protein','Alkaline_phosphatase']
biologicalAge_dict = dict(zip(biologicalAge_names, biologicalAge_fields))

In [19]:
KDM_BA_biomarkers = ['FEV1', 'SBP', 'Albumin', 
                     'Alkaline_phosphatase', 'Blood_urea_nitrogen','Creatinine', 
                     'C-reactive_protein', 'Glycated_hemoglobin','Cholesterol'] 
KDM_BA_biomarkers_left = ['Albumin', 
                     'Alkaline_phosphatase', 'Blood_urea_nitrogen','Creatinine', 
                     'C-reactive_protein', 'Glycated_hemoglobin','Cholesterol']
                  
phenoAge_biomarkers =['Albumin', 'Alkaline_phosphatase', 'Creatinine', 
                      'C-reactive_protein', 'Glucose', 'Mean_cell_volume', 
                      'Red_cell_distribution_width', 'White_blood_cell_count','Lymphocyte']

In [20]:
KDM_BA_fields_left = [biologicalAge_dict[bm] for bm in KDM_BA_biomarkers_left]
print(KDM_BA_fields_left)
phenoAge_fields = [biologicalAge_dict[bm] for bm in phenoAge_biomarkers]
print(phenoAge_fields)

['30600', '30610', '30670', '30700', '30710', '30750', '30690']
['30600', '30610', '30700', '30710', '30740', '30270', '30070', '30000', '30180']


In [21]:
pattern = r'^p(' + '|'.join(KDM_BA_fields_left) + r')_i0(_a\d+)?$'
KDM_BA_df = data_df.filter(regex=pattern).copy()

In [22]:
KDM_BA_df.columns = KDM_BA_biomarkers_left
KDM_BA_df['eid'] = data_df['eid']
KDM_BA_df['age'] = data_df.filter(like='21003_i0')

In [23]:
KDM_BA_df['FEV1'] = data_df[['p3063_i0_a0', 'p3063_i0_a1', 'p3063_i0_a2']].max(axis=1, skipna=True)
KDM_BA_df['SBP'] = data_df[['p4080_i0_a0', 'p4080_i0_a1']].mean(axis=1, skipna=True)

In [27]:
kdm_result = kdm_calc(KDM_BA_df, KDM_BA_biomarkers)
df_kdm = kdm_result['data'][['eid', 'kdm_ba', 'kdm_acceleration']]
data_df = data_df.merge(df_kdm, on='eid', how='left')

In [25]:
pattern = r'^p(' + '|'.join(phenoAge_fields) + r')_i0(_a\d+)?$'
phenoAge_df = data_df.filter(regex=pattern).copy()
phenoAge_df.columns = phenoAge_biomarkers

In [26]:
phenoage_result = phenoage_calc(phenoAge_df, phenoAge_biomarkers, orig=True)

ValueError: Data must contain 'age', 'time', 'status', and all specified biomarkers